#### Metric Calculator
##### Author : Kunal Kalwankar

In [ ]:
# imports and config
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/cnn_dailymail')
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install spacy
import spacy

In [ ]:
# Import the tag files for clusters and files that signify testids

d1 = pd.read_csv('t5small_knn5_5epochs_0_.csv')
d2 = pd.read_csv('t5small_knn5_5epochs_1_.csv')
d3 = pd.read_csv('t5small_knn5_5epochs_2_.csv')
d4 = pd.read_csv('t5small_knn5_5epochs_3_.csv')
d5 = pd.read_csv('t5small_knn5_5epochs_4_.csv')

t1 = pd.read_csv('testids_1_to_5.csv')
t2 = pd.read_csv('testids_2_to_5.csv')
t3 = pd.read_csv('testids_3_to_5.csv')
t4 = pd.read_csv('testids_4_to_5.csv')
t5 = pd.read_csv('testids_5_to_5.csv')

d = pd.concat([d1,d2,d3,d4,d5])
t = pd.concat([t1,t2,t3,t4,t5])
data = d.copy()

In [ ]:
#Filter only for testids and required columns
data1 = data[data['id'].isin(list(t['TESTIDS']))]
data1.shape
data1 = data1[['article','highlights','predictions']]
x = data1.copy()
x = x[['predictions','highlights','article']]

In [ ]:
# pruner function removes the brackets '[]' and inverted commas since simpleT5 generates in a list format
def pruner(obj):
  return obj[2:][:-2]
x['predictions'] = x['predictions'].apply(pruner)
print(x.head())
data = x.copy()

In [ ]:
#SPACY TEST OUTPUT
nlp = spacy.load('en_core_web_sm')
sentence = "Apple is looking at buying a U.K. startup for $1 billion"
doc = nlp(sentence)
for ent in doc.ents:
    print(ent.text)

In [ ]:
data['article'] = data['article'].str.lower()
data['highlights'] = data['highlights'].str.lower()
data['predictions'] = data['predictions'].str.lower()

In [ ]:
#ENTITY HALLUCAINATION FUNCTIONS

def entity_filter(text):
  doc = nlp(text)
  entity_list = [ent.text for ent in doc.ents]
  return entity_list

def entity_hallucination(text,summ):
  metric_counter = []
  t = [entity_filter(i) for i in text]
  s = [entity_filter(k) for k in summ]

  for n,i in enumerate(t[:10]):
    ent_count = len(t[n])
    false_ent_count = [j for j in s[n] if j not in t[n]]
    flase_ent_count = len(ent_correct_count)
    try:
      x = (false_ent_count/ent_count)
    except:
      x=0
    metric_counter.append(x)
  
  try:
    return sum(metric_counter)/len(metric_counter)
  except:
    return 0

In [ ]:
#ROUGE METRICS

!pip install rouge_score
!pip install datasets
!pip install datasets
from datasets import load_metric
metric = load_metric("rouge")

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
# ENTITY HALLUCINATION WITH REPECT TO GOLDER SUMMARY
print(entity_hallucination(data['highlights'],data['predictions']))

# ENTITY HALLUCINATION WITH REPECT TO ARTICLE
print(entity_hallucination(data['article'],data['predictions']))

#CALCULATE ROUGE SCORES
print(calc_rouge_scores(list(data['predictions']), list(data['highlights'])))